## Bosques de Aislamiento

In [1]:
import time
import pandas as pd
from itertools import combinations
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, max_error, r2_score
from sklearn.ensemble import IsolationForest

# Cargar datos desde el archivo CSV
df_datos = pd.read_csv("datos/insurance.csv")

# Codificación One Hot para las columnas categóricas
columnas_categoricas = ['sex', 'smoker', 'region']
codificador_one_hot = OneHotEncoder(sparse=False)
codificacion_one_hot = codificador_one_hot.fit_transform(df_datos[columnas_categoricas])
arr_nombre_nuevas_columnas = codificador_one_hot.get_feature_names_out(columnas_categoricas)
df_nuevas_columnas_one_hot = pd.DataFrame(codificacion_one_hot, columns=arr_nombre_nuevas_columnas)
df_datos = pd.concat([df_datos, df_nuevas_columnas_one_hot], axis=1)

# Eliminar las columnas originales categóricas
df_datos.drop(columns=columnas_categoricas, inplace=True)

# Columnas incluyendo las codificadas con one-hot
columnas_especificas = df_datos.columns.tolist()

# Crear conjunto de datos solo con las columnas seleccionadas
df_X = pd.DataFrame(df_datos, columns=columnas_especificas)
df_y = pd.DataFrame(df_datos, columns=['charges'])

# Detección y eliminación de anomalías utilizando Bosques de Aislamiento
detector_anomalias = IsolationForest(contamination=0.05, random_state=100)
tiempo_inicio_anomalias = time.time()
df_X['anomalia'] = detector_anomalias.fit_predict(df_X)
tiempo_fin_anomalias = time.time()
tiempo_elim_anomalias = tiempo_fin_anomalias - tiempo_inicio_anomalias

# Filtrar los datos para eliminar anomalías
df_X_clean = df_X[df_X['anomalia'] != -1].copy()
df_y_clean = df_y.loc[df_X_clean.index]

# Eliminar la columna de anomalías
df_X_clean.drop(columns=['anomalia'], inplace=True)

# Llenar valores faltantes con SimpleImputer solo en las columnas numéricas
imputer = SimpleImputer(strategy='mean')
df_X_filled = pd.DataFrame(imputer.fit_transform(df_X_clean), columns=df_X_clean.columns)

# Normalizar variables independientes usando MinMaxScaler
scaler_X = MinMaxScaler()
df_X_scaled = pd.DataFrame(scaler_X.fit_transform(df_X_filled), columns=df_X_filled.columns)

# Dividir conjunto de datos en entrenamiento y prueba
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X_scaled, df_y_clean, test_size=0.2, random_state=100)

# Lista para almacenar los resultados
resultados_knn = []

# Valores de k para KNN
valores_k = [1, 2, 3]

# Iterar sobre los valores de k
for k in valores_k:
    # Generar todas las combinaciones posibles de columnas
    for i in range(1, len(columnas_especificas) + 1):
        combinaciones_columnas = combinations(columnas_especificas, i)
        for combo in combinaciones_columnas:
            # Convertir la combinación en una lista para poder seleccionar las columnas del DataFrame
            combo_lista = list(combo)
            
            # Generar el modelo de regresión KNN y medir el tiempo de ejecución del modelo
            tiempo_inicio_modelo = time.time()
            model = KNeighborsRegressor(n_neighbors=k)
            model.fit(df_X_train[combo_lista], df_y_train.values.ravel())
            tiempo_fin_modelo = time.time()
            tiempo_modelo = tiempo_fin_modelo - tiempo_inicio_modelo
            
            # Realizar la prueba del modelo y medir el tiempo de ejecución de la prueba
            tiempo_inicio_prueba = time.time()
            arr_y_test_predicho = model.predict(df_X_test[combo_lista])
            tiempo_fin_prueba = time.time()
            tiempo_prueba = tiempo_fin_prueba - tiempo_inicio_prueba

            # Métricas de evaluación del modelo
            max_ea = max_error(df_y_test, arr_y_test_predicho)
            mae = mean_absolute_error(df_y_test, arr_y_test_predicho)
            mse = mean_squared_error(df_y_test, arr_y_test_predicho)
            r2 = r2_score(df_y_test, arr_y_test_predicho)

            # Almacenar resultados en la lista
            resultados_knn.append({
                'Valor_K': k,
                'Columnas': combo_lista,
                'Máximo_EA': max_ea,
                'EA_Medio': mae,
                'R^2': r2,
                'Tiempo_Modelo': tiempo_modelo,
                'Tiempo_Prueba': tiempo_prueba
            })

# Calcular el tiempo total de preprocesamiento
tiempo_preprocesamiento_total = tiempo_elim_anomalias
print(f"Tiempo de preprocesamiento total: {tiempo_preprocesamiento_total:.4f} segundos")
print("-" * 90)

# Crear un DataFrame a partir de la lista de resultados
df_resultados_knn = pd.DataFrame(resultados_knn)

# Imprimir resultados
for idx, row in df_resultados_knn.iterrows():
    print(f"Resultados para K={row['Valor_K']} y Columnas={row['Columnas']}:")
    print(f"   Máximo EA: {row['Máximo_EA']:.2f}")
    print(f"   EA Medio: {row['EA_Medio']:.2f}")
    print(f"   R^2: {row['R^2']:.4f}")
    print(f"   Tiempo de Generación del Modelo: {row['Tiempo_Modelo']:.4f} segundos")
    print(f"   Tiempo de Prueba del Modelo: {row['Tiempo_Prueba']:.4f} segundos")
    print("-" * 90)


C:\Users\rimel\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Tiempo de preprocesamiento total: 0.1638 segundos
------------------------------------------------------------------------------------------
Resultados para K=1 y Columnas=['age']:
   Máximo EA: 34985.06
   EA Medio: 9457.18
   R^2: -1.2093
   Tiempo de Generación del Modelo: 0.0020 segundos
   Tiempo de Prueba del Modelo: 0.0010 segundos
------------------------------------------------------------------------------------------
Resultados para K=1 y Columnas=['bmi']:
   Máximo EA: 45437.08
   EA Medio: 10848.34
   R^2: -1.2270
   Tiempo de Generación del Modelo: 0.0010 segundos
   Tiempo de Prueba del Modelo: 0.0020 segundos
------------------------------------------------------------------------------------------
Resultados para K=1 y Columnas=['children']:
   Máximo EA: 37699.33
   EA Medio: 10981.29
   R^2: -1.4049
   Tiempo de Generación del Modelo: 0.0025 segundos
   Tiempo de Prueba del Modelo: 0.0020 segundos
----------------------------------------------------------------------

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Resultados para K=3 y Columnas=['bmi', 'charges', 'sex_female', 'smoker_no', 'smoker_yes', 'region_northeast', 'region_southeast']:
   Máximo EA: 9096.53
   EA Medio: 969.73
   R^2: 0.9729
   Tiempo de Generación del Modelo: 0.0020 segundos
   Tiempo de Prueba del Modelo: 0.0025 segundos
------------------------------------------------------------------------------------------
Resultados para K=3 y Columnas=['bmi', 'charges', 'sex_female', 'smoker_no', 'smoker_yes', 'region_northeast', 'region_southwest']:
   Máximo EA: 9096.53
   EA Medio: 833.52
   R^2: 0.9806
   Tiempo de Generación del Modelo: 0.0020 segundos
   Tiempo de Prueba del Modelo: 0.0030 segundos
------------------------------------------------------------------------------------------
Resultados para K=3 y Columnas=['bmi', 'charges', 'sex_female', 'smoker_no', 'smoker_yes', 'region_northwest', 'region_southeast']:
   Máximo EA: 8413.26
   EA Medio: 983.49
   R^2: 0.9752
   Tiempo de Generación del Modelo: 0.0020 segundos